<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| no. Paquete | T. llegada | T. salida | Tiempo de arribo | Inicio de servicio | Hora de salida | Tiempo en fila | Tiempo en el sistema |
|-------------|------------|-----------|-------------------|---------------------|-----------------|----------------|------------------------|
1| 2

In [ ]:
#bruh